In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import BaseOutputParser
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceHub
from langchain_core.prompts import PromptTemplate
from langchain_classic import LLMChain
# from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain_classic.chains import SimpleSequentialChain, SequentialChain

OpenAI

In [ ]:
llmo = OpenAI(model='gpt-3.5-turbo-instruct' ,temperature=0.7)
llmo.invoke("What would be a good company name for a company that makes colorful socks?")

Prompt Template and LLMChain

In [ ]:
prompt_template = PromptTemplate(input_variables=["country"], 
                                 template="Tell me the capital of {country}")
prompt_template.format(country="France")

In [ ]:
chain = LLMChain(llm=llmo, prompt=prompt_template)
chain.run("Germany")

Combining Multiple Chains Using simple Sequential Chain

In [ ]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of {country}")
capital_chain = LLMChain(llm=llmo, prompt=capital_prompt)
famous_template = PromptTemplate(input_variables=['capital'],
                                 template='suggest palces to visit in {capital}')
famous_chain = LLMChain(llm=llmo, prompt=famous_template)

In [ ]:
chain_seq = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain_seq.run("Germany")

Sequential Chain

In [ ]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of {country}")
capital_chain = LLMChain(llm=llmo, prompt=capital_prompt, output_key='capital')

famous_template = PromptTemplate(input_variables=['capital'],
                                 template='suggest palces to visit in {capital}')
famous_chain = LLMChain(llm=llmo, prompt=famous_template, output_key='places')

In [ ]:
chain_seq = SequentialChain(chains=[capital_chain, famous_chain], 
                            input_variables=['country'],
                            output_variables=['capital', 'places'])
chain_seq({"country":"Germany"})

Chatmodels with ChatOpenAI

In [ ]:
chatllm = ChatOpenAI(model='gpt-3.5-turbo' ,temperature=0.7)

In [ ]:
chatllm([
    SystemMessage(content='You are a langchain expert'),
    HumanMessage(content='Return only langchain retaled')
])

Prompt Template + LLM +  Output Parsers

In [ ]:
class Commasepratedoutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")
    

In [ ]:
template = "You are a helpfule assistant. When user give input generate 10 words in a comma seprated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [ ]:
chain = chatprompt | chatllm | Commasepratedoutput()
chain.invoke({"text":"programming languages"})

## Reasoning models

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("")
])

# Langchain Messages 

### Message Objects

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="You are a assistant."),
    HumanMessage(content="Make a Langchain agent."),
    AIMessage(content="Sure, here is the agent")
]

### Tuple Format

In [ ]:
messages = [
    ("system","You are a assistant."),
    ("human","Make a Langchain agent."),
    ("ai","Sure, here is the agent")
]

### Dictionary Format

In [ ]:
messages = [
    {"role": "system", "content":"You are a assistant."},
    {"role": "user", "content":"Make a Langchain agent."},
    {"role": "assistant", "content":"Sure, here is the agent"}
]

### ChatPromptTemplate with Placeholders

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful {assistant_type}"),
    ("human", "Write a {language} function to calculate {function_name}")
])

messages = template.format_messages(
    assistant_type="programming assistant",
    language="Python",
    function_name="factorial"
)

### Message Prompt Template

In [ ]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

system_template = SystemMessagePromptTemplate.from_template(
    "You are a helpful {assistant_type}"
)
human_template = HumanMessagePromptTemplate.from_template(
    "Write a {language} function to calculate {function_name}"
)

chat_prompt = ChatPromptTemplate.from_messages([
    system_template, 
    human_template
])

messages = chat_prompt.format_prompt(
    assistant_type="programming assistant",
    language="Python", 
    function_name="factorial"
).to_messages()

### Using Message History

In [ ]:
from langchain_core.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("Write a Python function to calculate factorial")
history.add_ai_message("Here's a Python function for factorial...")

messages = history.messages

### prompt used in production

In [ ]:
from langchain_core.prompts import PromptTemplate

# Define once, reuse everywhere
question_template = PromptTemplate.from_template( "Answer this question concisely: {question}" )
question_with_context_template = PromptTemplate.from_template("Context information: {context}\n\nAnswer this question concisely:{question}" )
# Generate prompts by filling in variables
prompt_text = question_template.format(question="What is the capitalof France?")

### chat prompt template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
template = ChatPromptTemplate.from_messages([
    ("system", "You are an English to French translator."),
    ("user", "Translate this to French: {text}")
])
chat = ChatOpenAI()
formatted_messages = template.format_messages(text="Hello, how are you?")
response = chat.invoke(formatted_messages)
print(response.content)

chain = RunnableSequence(first= prompt, middle=[llm], last= output_parser)

LCEL also supports adding transformations and custom functions:

with_transformation = prompt | llm | (lambda x: x.upper()) |
StrOutputParser()

In [ ]:
# branch logic

decision_chain = prompt | llm | (lambda x: route_based_on_content(x)) | {
    "summarize": summarize_chain,
    "analyze": analyze_chain
}

In [ ]:
# Function to Runnable
length_func = lambda x: len(x)

chain = prompt | length_func | output_parser

# Is converted to:
chain = prompt | RunnableLambda(length_func) | output_parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Create components
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
llm = ChatOpenAI()
output_parser = StrOutputParser()

# Chain them together using LCEL
chain = prompt | llm | output_parser

#Execute the workflow with a single call
result = chain.invoke({"topic": "programming"})
print(result)

### Complex chain example

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# Initialize the model
llm = GoogleGenerativeAI(model="Gemini 2.5 Pro	")

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about{topic}")
story_chain = story_prompt | llm | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template("Analyze the following story's mood:\n{story}")
analysis_chain = analysis_prompt | llm | StrOutputParser()

In [ ]:
# Combine chains
story_with_analysis = story_chain | analysis_chain

# Run the combined chain
story_analysis = story_with_analysis.invoke({"topic": "a rainy day"})
print("\nAnalysis:", story_analysis)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
# Using RunnablePassthrough.assign to preserve data
# Add 'story' key with generated content

enhanced_chain = RunnablePassthrough.assign(story=story_chain).assign(analysis=analysis_chain)

# Execute the chain
result = enhanced_chain.invoke({"topic": "a rainy day"})
print(result.keys()) # Output: dict_keys(['topic', 'story', 'analysis'])
# dict_keys(['topic', 'story', 'analysis'])

In [ ]:
from operator import itemgetter
# Alternative approach using dictionary construction
manual_chain = (RunnablePassthrough() |
# Pass through input
    {
        "story": story_chain,
        # Add story result
        "topic": itemgetter("topic")
# Preserve original topic
    } |
    RunnablePassthrough().assign(
# Add analysis based on story
    analysis=analysis_chain
)
)
result = manual_chain.invoke({"topic": "a rainy day"})
print(result.keys())
# Output: dict_keys(['story', 'topic', 'analysis'])

In [ ]:
# Simplified dictionary construction
simple_dict_chain = story_chain | {"analysis": analysis_chain}
result = simple_dict_chain.invoke({"topic": "a rainy day"}) 
print(result.keys()) # Output: dict_keys(['analysis', 'output'])